In [26]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler


In [ ]:
# Load dataset
df = pd.read_csv("electricvehicles.csv")

# Drop unused column
df = df.drop(columns=["Segment"])

# Clean and preprocess
df = df.dropna(subset=[
    "AccelSec", "TopSpeed_KmH", "Range_Km", "Efficiency_WhKm",
    "FastCharge_KmH", "RapidCharge", "Seats", "PriceEuro"
])

# Encode categorical values
powertrain_map = {"AWD": 1.0, "RWD": 0.75, "FWD": 0.5}
df["PowerTrainScore"] = df["PowerTrain"].map(powertrain_map).fillna(0.5)
df["RapidChargeScore"] = df["RapidCharge"].apply(lambda x: 1 if str(x).lower() == "yes" else 0)

# Select numerical columns for scoring (excluding PriceEuro from score)
"""score_columns = {
    "AccelSec": "higher",
    "TopSpeed_KmH": "higher",
    "Range_Km": "higher",
    "Efficiency_WhKm": "lower",
    "FastCharge_KmH": "lower",
    "Seats": "lower",
    "RapidChargeScore": "higher",
    "PowerTrainScore": "lower"
}"""

score_columns = {
    "AccelSec": "lower",
    "TopSpeed_KmH": "higher",
    "Range_Km": "higher",
    "Efficiency_WhKm": "lower",
    "FastCharge_KmH": "higher",
    "Seats": "higher",
    "RapidChargeScore": "higher",
    "PowerTrainScore": "higher"
}

'\nscore_columns = {\n    "AccelSec": "lower",\n    "TopSpeed_KmH": "higher",\n    "Range_Km": "higher",\n    "Efficiency_WhKm": "lower",\n    "FastCharge_KmH": "higher",\n    "Seats": "higher",\n    "RapidChargeScore": "higher",\n    "PowerTrainScore": "higher"\n}\n\n'

In [28]:
# Normalize numerical features
scaler = MinMaxScaler()
norm_df = df[[*score_columns]].copy()
norm_df
norm_df.to_csv('norm_df.csv')

In [ ]:
for col, direction in score_columns.items():
    scaled = scaler.fit_transform(norm_df[[col]])
    norm_df[col] = 1 - scaled if direction == "lower" else scaled

In [ ]:
# Compute raw score (excluding price)
norm_df["RawValueScore"] = norm_df.mean(axis=1)

# Adjust score based on price to get value per euro
df["ValueScore"] = norm_df["RawValueScore"] / df["PriceEuro"]

# Sort and show top result
top_vehicle = df.sort_values("ValueScore", ascending=False).iloc[0]
lowest_vehicle = df.sort_values("ValueScore", ascending=True).iloc[0]


# Output top vehicle details
print("Top Value-per-Euro Vehicle:")
print(top_vehicle[["Brand", "Model", "ValueScore"]])
print("\nLowest Value-per-Euro Vehicle:")
print(lowest_vehicle[["Brand", "Model", "ValueScore"]])

df.sort_values("ValueScore", ascending=False)

Top Value-per-Euro Vehicle:
Brand                                 Tesla
Model         Model Y Long Range Dual Motor
ValueScore                         0.000011
Name: 21, dtype: object

Lowest Value-per-Euro Vehicle:
Brand                Porsche
Model         Taycan Turbo S
ValueScore          0.000003
Name: 16, dtype: object


,Brand,Model,AccelSec,TopSpeed_KmH,Range_Km,Efficiency_WhKm,FastCharge_KmH,RapidCharge,PowerTrain,PlugType,BodyStyle,Seats,PriceEuro,PowerTrainScore,RapidChargeScore,ValueScore
21,Tesla,Model Y Long Range Dual Motor,5.1,217,425,171,930.0,Yes,AWD,Type 2 CCS,SUV,7,58620,1.00,1,0.000011
0,Tesla,Model 3 Long Range Dual Motor,4.6,233,450,161,940.0,Yes,AWD,Type 2 CCS,Sedan,5,55480,1.00,1,0.000010
26,Volkswagen,ID.3 Pro,9.0,160,350,166,490.0,Yes,RWD,Type 2 CCS,Hatchback,5,33000,0.75,1,0.000010
39,Mercedes,EQA,5.0,200,350,171,440.0,Yes,AWD,Type 2 CCS,SUV,5,45000,1.00,1,0.000010
61,Tesla,Model Y Long Range Performance,3.7,241,410,177,900.0,Yes,AWD,Type 2 CCS,SUV,7,65620,1.00,1,0.000010
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48,Lightyear,One,10.0,150,575,104,540.0,Yes,AWD,Type 2 CCS,Liftback,5,149000,1.00,1,0.000003
51,Tesla,Roadster,2.1,410,970,206,920.0,Yes,AWD,Type 2 CCS,Cabrio,4,215000,1.00,1,0.000003
84,Mercedes,EQV 300 Long,10.0,140,330,273,290.0,Yes,FWD,Type 2 CCS,SPV,7,70631,0.50,1,0.000003
72,Porsche,Taycan Cross Turismo,3.5,250,385,217,770.0,Yes,AWD,Type 2 CCS,Station,4,150000,1.00,1,0.000003
